In [1]:
import tensorcircuit as tc
import tensorflow as ts 
import numpy as np
import math
from numpy import linalg as LA
K = tc.set_backend("tensorflow")

sim_t = math.pi
interval = 10000
dt = sim_t / interval
cur_t = 0 
h_m = 0.5

qubit = 2
min_w = 999
w=0
v=0
I = tc.gates._i_matrix
X = tc.gates._x_matrix  
Y = tc.gates._y_matrix  
Z = tc.gates._z_matrix

In [2]:
def i_iterate_multiplication(u):
    tmp = 1
    for i in range(u):
        tmp = np.kron(I, tmp)
    return tmp

def zzi(i):
     return np.kron(np.kron(i_iterate_multiplication(i), np.kron(Z, Z)), i_iterate_multiplication(qubit-2-i))
    
def xi(i):
    return np.kron(np.kron(i_iterate_multiplication(i), X), i_iterate_multiplication(qubit-1-i))

h_Z = 0
h_X = 0
for i in range(qubit-1):
    h_Z = h_Z - zzi(i)
for i in range(qubit):
    h_X = h_X + h_m * xi(i)
def evolution(h, t):
    return tc.gates.exponential_gate(h, t)

In [4]:
#w, v = LA.eig(H)
#for i in range(0, len(w)):
#        if w[i] < min_w:
#            min_w = w[i]

c_precise=tc.Circuit(qubit)
c_trotter=tc.Circuit(qubit)
u_h = evolution(h_X+h_Z, sim_t)
u_X = evolution(h_X, dt)
u_Y = evolution(h_Z, dt)
c_precise.any(*range(qubit), unitary=evolution(h_X+h_Z, sim_t))
print(c_precise.state())
for _interval in range(interval):
    c_trotter.any(*range(qubit), unitary=evolution(h_X, dt))
    c_trotter.any(*range(qubit), unitary=evolution(h_Z, dt))
    cur_t += dt
print(c_trotter.state())

tf.Tensor(
[-6.3312793e-01-0.34079105j  2.5527278e-09+0.340791j
  1.4097814e-08+0.34079096j  3.6687195e-01-0.3407909j ], shape=(4,), dtype=complex64)
tf.Tensor(
[-6.3280398e-01-0.34062576j  1.2966990e-04+0.3406784j
  1.2699515e-04+0.34067908j  3.6669871e-01-0.3406357j ], shape=(4,), dtype=complex64)
